# Trabalho sobre a aplicação do método variacional.

**Questão 1**.

Na aula 2, vimos que o **método variacional** é uma técnica aproximativa eficaz para estimar a energia do estado fundamental do átomo de hélio.
Agora, considere novamente o átomo de hélio e utilize o método variacional para **estimar a energia do primeiro estado excitado**.

**Questão 2**.

Ainda considerando o átomo de hélio, **calcule a frequência do fóton emitido** na transição do **primeiro estado excitado** para o **estado fundamental**.

 

**Questão 3**.

Na aula 5, vimos como aplicar o método variacional em computação quântica para estimar a energia do estado fundamental da molécula de hidrogênio ($H_2$). Observamos que, para cada geometria possível da molécula, é necessário usar o VQE (Variational Quantum Eigensolver) para obter o menor valor médio $\langle H \rangle$. Nesta atividade, vamos considerar uma abordagem mais avançada, em que a **geometria da molécula também é tratada como um problema de otimização**[1](https://arxiv.org/abs/2106.13840). O **código parcial fornecido ao final da questão** aplica o VQE ao estudo do $H_2$, otimizando tanto os parâmetros do circuito quanto a geometria da molécula. Ou seja, o código busca a **geometria de menor energia** que **estabiliza** a molécula de hidrogênio.

Tarefas:

1) Execute o código fornecido no Google Colab e utilize-o para **obter a melhor geometria possível** da molécula de hidrogênio.

2) Compare a geometria obtida ao final da simulação com a melhor geometria discutida na aula 5.
- Elas são iguais ou diferentes?
- Explique a possível causa da semelhança ou diferença.

3) Por que o estado inicial é $|1100\rangle$?

4) Qual foi o motivo da escolha da parametrização $U$ usada no circuito variacional?

5) Podemos usar o seguinte estado de teste?

$$|\psi \rangle = \alpha_0|0000\rangle + \alpha_1|0001\rangle + \alpha_2|0011\rangle + \alpha_3|0101\rangle.
$$

- Justifique.

6) Após obter a melhor geometria, **compare o valor médio $\langle H \rangle$** com aquele obtido na aula 5.

- Os valores são iguais?

- Se forem diferentes, qual seria uma possível explicação? (Dica: Erros numéricos e tolerâncias de convergência podem afetar os resultados.)

**OBS**: Para rodar o código no Google Colab, instale o PennyLane com o seguinte comando no início da célula: ``!pip install pennylane``

----------------

##### Espaço para feedback – Avaliação das aulas (Resposta opcional)

Gostaria que você escrevesse, a partir do seu ponto de vista, o que achou das aulas ministradas. Conte o que gostou, o que funcionou bem e, principalmente, o que você acha que poderia ser melhorado.

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

qubits = 'Defina aqui o número de qubits necessários'

dev = qml.device("default.qubit", wires=qubits)

@qml.qnode(dev)
def Circuit(x, theta):
    symbols = ["H", "H"]
    
    H = qml.qchem.molecular_hamiltonian(symbols, x)[0]
    
    electrons = 2
    hf = qml.qchem.hf_state(electrons, qubits)
    
    qml.BasisState(hf, wires=range(qubits))
    '''
    Defina aqui a parametrização U que você acha que deve ser usada
    '''
    
    return qml.expval(H)


def Grad_X(x, theta, eta, function, delta):
    dx = np.zeros(x.shape)
    f1 = function(x,theta)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x1 = x.copy()
            x1[i][j] += delta
            f2 = function(x1,theta)
            dx[i][j] = (f2-f1)/delta
    return dx   


np.random.seed(42)
x = np.random.random((2,3))#Gera uma geometria inicial aleatória 

theta = np.random.random('Aqui deve ser informado quantos parâmetros U vai ter')

eta = 'Defina aqui a taxa de aprendizado'
epocas = 'Defina aqui o número de épocas'
delta = 'Defina aqui aqui o valor de detal usada para a diferenciação'


opt = qml.GradientDescentOptimizer(stepsize=eta)

energia = []
for i in trange(epocas):
    #calcula o valor medio
    energia.append( Circuit(x,theta) )
    
    #gradiente de x
    dx = Grad_X(x, theta, eta, Circuit, delta)
    
    #gradiente de theta
    theta = opt.step(lambda theta: Circuit(x,theta), theta)
    
    # atualiza x
    x = x - eta*dx

    
    
y = np.array(energia)
z = np.arange(len(y))
plt.plot(z,y)
plt.ylabel(r'\langle H \rangle')
plt.xlabel('Épocas')
plt.show()